## Import

In [1]:
import pandas as pd
import numpy as np
import random
import os

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader 

from tqdm.auto import tqdm

c:\ProgramData\anaconda3\envs\umpy2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Fixed Random Seed

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42) # Seed 고정

## Data Load

In [3]:
train_df = pd.read_csv('C:\Workspace\power_consumption_comp\data/train.csv')
test_df = pd.read_csv('C:\Workspace\power_consumption_comp\data/test.csv')
sample_submission = pd.read_csv('C:\Workspace\power_consumption_comp\data/sample_submission.csv')

In [4]:
train_df.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40


## Train Data Pre-processing

In [5]:
# 일조, 일사 열 제거
train_df = train_df.drop(['일조(hr)','일사(MJ/m2)'], axis=1)
train_df.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,986.40


In [6]:
# 결측치 확인
train_df.isna().sum()

num_date_time         0
건물번호                  0
일시                    0
기온(C)                 0
강수량(mm)          160069
풍속(m/s)              19
습도(%)                 9
전력소비량(kWh)            0
dtype: int64

In [7]:
# 강수량 결측치 0.0으로 채우기
train_df['강수량(mm)'].fillna(0.0, inplace=True)

# 풍속, 습도 결측치 평균으로 채우고 반올림하기
train_df['풍속(m/s)'].fillna(round(train_df['풍속(m/s)'].mean(),2), inplace=True)
train_df['습도(%)'].fillna(round(train_df['습도(%)'].mean(),2), inplace=True)

In [8]:
train_df['month'] = train_df['일시'].apply(lambda x : float(x[4:6]))
train_df['day'] = train_df['일시'].apply(lambda x : float(x[6:8]))
train_df['time'] = train_df['일시'].apply(lambda x : float(x[9:11]))

In [9]:
# 순서 재배치
train_df = train_df[train_df.columns[:7].to_list() + train_df.columns[8:].to_list() + train_df.columns[7:8].to_list()]
train_df.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),month,day,time,전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,0.0,0.9,42.0,6.0,1.0,0.0,1085.28
1,1_20220601 01,1,20220601 01,18.0,0.0,1.1,45.0,6.0,1.0,1.0,1047.36
2,1_20220601 02,1,20220601 02,17.7,0.0,1.5,45.0,6.0,1.0,2.0,974.88
3,1_20220601 03,1,20220601 03,16.7,0.0,1.4,48.0,6.0,1.0,3.0,953.76
4,1_20220601 04,1,20220601 04,18.4,0.0,2.8,43.0,6.0,1.0,4.0,986.40


## Hyperparameter Setting

In [10]:
# 하이퍼파라미터
input_size = 24  # feature의 개수
hidden_size = 64  # 은닉층의 크기
num_layers = 3  # LSTM 층의 개수
output_size = 1  # 출력의 크기
num_epochs = 15  # 학습 횟수
window_size = 24  # 예측에 사용될 시간 윈도우 크기
batch_size = 1200  # 배치 크기
learning_rate = 0.0001  # 학습률

## Dataset

In [11]:
class TimeSeriesDataset(Dataset):
    def __init__(self, df, window_size):
        self.df = df
        self.window_size = window_size

    def __len__(self):
        return len(self.df) - self.window_size

    def __getitem__(self, idx):
        x = torch.tensor(self.df[idx:idx+self.window_size, :], dtype=torch.float)
        if self.df.shape[1] > 1:
            y = torch.tensor(self.df[idx+self.window_size, -1], dtype=torch.float)
        else:
            y = None
        return x, y

def create_data_loader(df, window_size, batch_size):
    dataset = TimeSeriesDataset(df, window_size)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    return data_loader

In [12]:
# normalization
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_df.drop(['num_date_time', '건물번호', '일시'], axis=1).values)
train_loader = create_data_loader(train_data, window_size, batch_size)

## Model Define

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class HybridCNNBiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(HybridCNNBiLSTM, self).__init__()
        
        # CNN parameters
        self.conv1 = nn.Conv1d(input_size, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
        # LSTM parameters
        self.hidden_dim = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(128, hidden_size, num_layers, batch_first=True, bidirectional=True)
        
        # Linear layer
        self.fc = nn.Linear(hidden_size*2, output_size)  # 2 for bidirection

    def forward(self, x):
        # CNN
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        
        # LSTM
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_dim).to(x.device)  # 2 for bidirection
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_dim).to(x.device)  # 2 for bidirection
        out, _ = self.lstm(x, (h0, c0))
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        
        return out


In [14]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.fc = nn.Linear(input_size, batch_size)  # 입력 차원을 입력 데이터의 피처 수로 설정
        self.lstm = nn.LSTM(batch_size, hidden_size, num_layers, batch_first=True)  # LSTM 레이어의 입력 차원을 Fully Connected 레이어의 출력 차원과 일치하게 설정
        
        

        self.fc_out = nn.Linear(hidden_size, output_size)  # 출력 레이어

    def forward(self, x):
        x = self.fc(x)  # 입력 데이터를 Fully Connected 레이어에 전달하여 차원을 늘림

        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # 초기 hidden state 생성
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # 초기 cell state 생성

        out, _ = self.lstm(x, (h0, c0))  # LSTM 레이어의 forward 연산 수행
        out = self.fc_out(out[:, -1, :])  # 마지막 시퀀스의 hidden state를 입력으로 Fully Connected 레이어에 전달

        return out


In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 현재 사용 가능한 디바이스를 확인하고, cuda가 사용 가능하면 cuda를 사용하고 그렇지 않으면 cpu를 사용한다.
print(f"current device: {device}")

model = LSTM(input_size, hidden_size, num_layers, output_size).to(device)  # LSTM 모델을 생성하고, 해당 모델을 위에서 정의한 디바이스에 할당한다.
#loss function을 CrossEntropy로 설정
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  # Adam 옵티마이저를 생성하고, 모델의 파라미터를 최적화한다. 학습률은 learning_rate로 설정한다.

current device: cuda


## Train

In [16]:
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)  # 입력 데이터를 디바이스로 이동
        labels = labels.unsqueeze(1).to(device)  # 레이블을 디바이스로 이동

        # Forward
        outputs = model(inputs)  # 모델에 입력 데이터를 전달하여 예측값 계산
        loss = criterion(outputs, labels)  # 예측값과 실제 레이블 간의 손실 계산

        # Backward and optimize
        optimizer.zero_grad()  # 기울기 초기화
        loss.backward()  # 손실에 대한 역전파 수행
        optimizer.step()  # 옵티마이저를 사용하여 모델 파라미터 업데이트

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (28800x8 and 24x1200)

## Test Data Pre-processing

In [17]:
# 학습 데이터에서 마지막 행 가져오기
last_train_data = train_df.drop(['num_date_time', '건물번호', '일시',], axis=1).loc[204000-24:,:]

# 실수형 데이터로 변환
test_df['습도(%)'] = test_df['습도(%)'].astype('float64')

# 날짜 데이터 추가
test_df['month'] = test_df['일시'].apply(lambda x : float(x[4:6]))
test_df['day'] = test_df['일시'].apply(lambda x : float(x[6:8]))
test_df['time'] = test_df['일시'].apply(lambda x : float(x[9:11]))

# 전력소비량 열 생성
final_df = pd.concat((test_df.drop(['num_date_time', '건물번호', '일시',], axis=1), pd.DataFrame(np.zeros(test_df.shape[0]))),axis=1)
final_df = final_df.rename({0:'전력소비량(kWh)'},axis=1)

## Test Dataset

In [18]:
test_df = pd.concat((last_train_data, final_df)).reset_index(drop=True)
test_data = scaler.transform(test_df.values) # train과 동일하게 scaling
test_data.shape

(16824, 8)

In [19]:
# Dataset & DataLoader
test_dataset = TimeSeriesDataset(test_data, window_size)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

## Inference

In [ ]:
model.eval()  # 모델을 평가 모드로 설정

test_predictions = []  # 테스트 예측 결과를 저장할 리스트 생성

with torch.no_grad():  # 그래디언트 계산 비활성화
    for i in range(test_data.shape[0] - window_size):  # 테스트 데이터의 길이에서 윈도우 크기를 뺀 범위에서 반복
        x = torch.Tensor(test_data[i:i+window_size,:]).to(device)  # 입력 데이터를 텐서로 변환하고 디바이스에 할당
        new_x = model(x.view(1,window_size,-1))  # 모델에 입력 데이터 전달하여 예측값 계산
        
        test_data[i+window_size,-1] = new_x  # 입력 데이터 업데이트
        test_predictions.append(new_x.detach().cpu().numpy().item())  # 예측 결과를 리스트에 추가

## Submit

In [ ]:
predictions = scaler.inverse_transform(test_data)[24:,-1] # 원래 scale로 복구

In [ ]:
sample_submission['answer'] = predictions
sample_submission.head()

,num_date_time,answer
0,1_20220825 00,556.266875
1,1_20220825 01,485.803986
2,1_20220825 02,421.522642
3,1_20220825 03,435.690004
4,1_20220825 04,524.522437


In [ ]:
sample_submission.to_csv('C:\Workspace\power_consumption_comp\data/lstm_baseline_submission.csv', index=False)